# Fixture Para Todos

## Valores parametrizables por el usuario

### Cantidad de equipos a realizarles intercambios

In [25]:
n_teams_to_be_swapped = 2

### Cantidad de intercambios a realizar por equipo

In [26]:
n_swaps_per_team = 2

### Cantidad de iteraciones del algoritmo

In [27]:
n_iterations = 100

In [28]:
heuristic_params = {"n_teams_to_be_swapped": n_teams_to_be_swapped,
                    "n_swaps_per_team": n_swaps_per_team,
                    "n_iterations": n_iterations}

## Parámetros de las restricciones

### Restricción 1

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante que el resto o, a lo sumo, <b>param_r1</b> partido/s de diferencia con respecto al resto de los equipos

In [29]:
param_r1 = 1

### Restricción 2

Todos los equipos deben jugar la misma cantidad de partidos de local y de visitante o, a lo sumo, <b>param_r2</b> partido/s de diferencia entre ambas condiciones

In [30]:
param_r2 = 1

### Restricción 3

Ningún equipo puede jugar más de <b>param_r3</b> partidos consecutivos en la misma condición

In [31]:
param_r3 = 2

### Restricción 4

Los equipos considerados clásicos entre sí no pueden jugar de local en la misma fecha

In [32]:
restrictions_params = {"r1": param_r1, "r2": param_r2, "r3": param_r3}

## Importación de librerías y archivos

In [33]:
import csv
import math
import time
from fixture_manager import FixtureManager
from distance_travelled_manager import DistanceTravelledManager

## Carga de archivos CSV

In [34]:
# Carga los equipos a partir del archivo csv
def load_teams(path):
    '''
        Formato de los equipos:
        ["Equipo A", ..., "Equipo Z"]
    '''

    teams = []
    with open(path) as f:
        content = f.readlines()
    teams = [x.strip() for x in content]

    return teams

In [35]:
# Carga las distancias a partir del archivo csv
def load_distances(path):
    '''
        Formato de las distancias:
        {"Equipo A": {"Equipo B": 100, ..., "Equipo Z": 200},
        ...,
        "Equipo X": {"Equipo Y": 150, ..., "Equipo Z": 300}
        }
    '''

    distances = {}
    header = True

    with open(path) as csv_file:
        csv_distances = csv.reader(csv_file, delimiter=",")
        for row in csv_distances:
            if header:
                header = False
                continue

            team1, team2, distance = row[0], row[1], row[2]

            if team1 not in distances:
                distances[team1] = {}
            distances[team1][team2] = distance

            if team2 not in distances:
                distances[team2] = {}
            distances[team2][team1] = distance

    return distances

In [36]:
# Carga los derbies a partir del archivo csv
def load_derbies(path):
    '''
    Formato de los derbies:
        {"Equipo A": "Equipo B",
        ...,
        "Equipo Y": "Equipo Z"
        }
    '''

    derbies = {}
    header = True

    with open(path) as csv_file:
        csv_derbies = csv.reader(csv_file, delimiter=",")
        for row in csv_derbies:
            if header:
                header = False
                continue
            
            team1 = row[0]
            team2 = row[1]
            derbies[team1] = team2
            derbies[team2] = team1

    return derbies

In [37]:
teams = load_teams("csv/8-teams/teams.csv")
distances = load_distances("csv/8-teams/distances.csv")
derbies = load_derbies("csv/8-teams/derbies.csv")

## Creación del Fixture Manager

In [38]:
fixture_manager = FixtureManager(teams, derbies, restrictions_params, distances, heuristic_params)

## Fixture Inicial

### Generación

In [39]:
start = time.time()
initial_fixture = fixture_manager.calculate_initial_fixture()
end = time.time()

Calculando fixture. Por favor espere...



In [40]:
print("Tiempo transcurrido: " + str(round(end - start, 2)) + " segundos")

Tiempo transcurrido: 0.11 segundos


In [41]:
fixture_manager.print(initial_fixture)

--------------------------------------------------------------
--------------------- FIXTURE PARA TODOS ---------------------
--------------------------------------------------------------

Fecha 1:
	Rosario Central		Vs.		Newells
	Boca Juniors		Vs.		Talleres
	Belgrano		Vs.		Union
	Colon			Vs.		River Plate

Fecha 2:
	Newells			Vs.		Talleres
	Boca Juniors		Vs.		Colon
	Belgrano		Vs.		River Plate
	Union			Vs.		Rosario Central

Fecha 3:
	Talleres		Vs.		Rosario Central
	Newells			Vs.		Union
	River Plate		Vs.		Boca Juniors
	Colon			Vs.		Belgrano

Fecha 4:
	Colon			Vs.		Talleres
	Rosario Central		Vs.		Boca Juniors
	Belgrano		Vs.		Newells
	River Plate		Vs.		Union

Fecha 5:
	Boca Juniors		Vs.		Newells
	Talleres		Vs.		Belgrano
	Rosario Central		Vs.		River Plate
	Union			Vs.		Colon

Fecha 6:
	Talleres		Vs.		Union
	Newells			Vs.		River Plate
	Boca Juniors		Vs.		Belgrano
	Colon			Vs.		Rosario Central

Fecha 7:
	Newells			Vs.		Colon
	Union			Vs.		Boca Juniors
	River Plate		Vs.		Talleres
	Belgrano		Vs

### Validación

In [42]:
print("¿Fixture compatible?")
fixture_manager.is_compatible(initial_fixture)

¿Fixture compatible?


True

### Cálculo de la varianza

In [43]:
fixture_manager.calculate_km_travelled_variance(initial_fixture)

Km recorridos por cada equipo:
- Newells: 723
- Talleres: 2203
- Union: 1356
- River Plate: 1733
- Colon: 653
- Rosario Central: 1159
- Boca Juniors: 807
- Belgrano: 1080

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:


1417.77

## Fixture Optimizado

In [44]:
start = time.time()
optimized_fixture = fixture_manager.calculate_optimized_fixture(initial_fixture)
end = time.time()

ITERACIÓN N° 1

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Colon: 720
- Boca Juniors: 1197
- Talleres: 1838
- Newells: 490
- River Plate: 1890
- Rosario Central: 1204
- Belgrano: 1177
- Union: 1198

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1271.45

ITERACIÓN N° 2

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Colon: 711
- River Plate: 1858
- Newells: 1010
- Talleres: 1875
- Belgrano: 1076
- Rosario Central: 1126
- Boca Juniors: 1266
- Union: 792

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1163.55

ITERACIÓN N° 3

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Rosario Central: 1049
- Belgrano: 1883
- River Plate: 2131
- Colon: 1212
- Newells: 597
- Boca Juniors: 648
- Talleres: 1096
- Union: 1098

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1429.72

ITERACIÓN N° 

Km recorridos por cada equipo:
- River Plate: 1454
- Talleres: 1573
- Colon: 990
- Newells: 899
- Boca Juniors: 1903
- Belgrano: 1104
- Rosario Central: 786
- Union: 1005

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1024.48

ITERACIÓN N° 27

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Talleres: 1487
- Newells: 1167
- Union: 896
- River Plate: 757
- Rosario Central: 629
- Boca Juniors: 2211
- Colon: 1100
- Belgrano: 1467

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1341.62

ITERACIÓN N° 28

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- River Plate: 2140
- Rosario Central: 916
- Colon: 835
- Belgrano: 1184
- Talleres: 1503
- Union: 1165
- Boca Juniors: 1197
- Newells: 774

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1170.74

ITERACIÓN N° 29

Calculando fixture. Por favor espere...

Km recorridos

Km recorridos por cada equipo:
- Union: 1064
- Boca Juniors: 1266
- Newells: 1188
- Talleres: 1145
- Belgrano: 2166
- Rosario Central: 715
- Colon: 544
- River Plate: 1626

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1343.35

ITERACIÓN N° 52

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Newells: 1282
- Boca Juniors: 1104
- Union: 699
- Belgrano: 1814
- River Plate: 1626
- Rosario Central: 474
- Talleres: 1507
- Colon: 1208

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1202.16

ITERACIÓN N° 53

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Colon: 720
- Belgrano: 1837
- Boca Juniors: 810
- Newells: 777
- Rosario Central: 991
- River Plate: 2143
- Union: 1295
- Talleres: 1141

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1382.39

ITERACIÓN N° 54

Calculando fixture. Por favor espere...

Km recorrido

Km recorridos por cada equipo:
- River Plate: 1890
- Newells: 1002
- Union: 1370
- Talleres: 1835
- Belgrano: 1489
- Rosario Central: 631
- Colon: 849
- Boca Juniors: 648

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1334.54

ITERACIÓN N° 77

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- Talleres: 1204
- Rosario Central: 879
- River Plate: 1167
- Colon: 1276
- Newells: 864
- Belgrano: 1768
- Union: 732
- Boca Juniors: 1824

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1073.41

ITERACIÓN N° 78

Calculando fixture. Por favor espere...

Km recorridos por cada equipo:
- River Plate: 2143
- Colon: 1180
- Belgrano: 1076
- Newells: 871
- Boca Juniors: 1045
- Union: 930
- Rosario Central: 896
- Talleres: 1573

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:
1157.63

ITERACIÓN N° 79

Calculando fixture. Por favor espere...

Km recorridos

In [45]:
print("Tiempo transcurrido: " + str(round(end - start, 2)) + " segundos")

Tiempo transcurrido: 16.5 segundos


In [46]:
fixture_manager.print(optimized_fixture)

--------------------------------------------------------------
--------------------- FIXTURE PARA TODOS ---------------------
--------------------------------------------------------------

Fecha 1:
	Belgrano		Vs.		Rosario Central
	Newells			Vs.		Talleres
	Boca Juniors		Vs.		Union
	Colon			Vs.		River Plate

Fecha 2:
	Belgrano		Vs.		Talleres
	Newells			Vs.		River Plate
	Boca Juniors		Vs.		Rosario Central
	Union			Vs.		Colon

Fecha 3:
	Union			Vs.		Newells
	River Plate		Vs.		Belgrano
	Rosario Central		Vs.		Colon
	Talleres		Vs.		Boca Juniors

Fecha 4:
	Colon			Vs.		Belgrano
	River Plate		Vs.		Rosario Central
	Talleres		Vs.		Union
	Newells			Vs.		Boca Juniors

Fecha 5:
	Union			Vs.		River Plate
	Rosario Central		Vs.		Talleres
	Boca Juniors		Vs.		Colon
	Belgrano		Vs.		Newells

Fecha 6:
	Rosario Central		Vs.		Union
	Colon			Vs.		Newells
	River Plate		Vs.		Talleres
	Belgrano		Vs.		Boca Juniors

Fecha 7:
	Union			Vs.		Belgrano
	Talleres		Vs.		Colon
	Newells			Vs.		Rosario Central
	River Plate	

### Validación

In [47]:
print("¿Fixture compatible?")
fixture_manager.is_compatible(optimized_fixture)

¿Fixture compatible?


True

### Cálculo de la varianza

In [48]:
fixture_manager.calculate_km_travelled_variance(optimized_fixture)

Km recorridos por cada equipo:
- Rosario Central: 1038
- Talleres: 1536
- Union: 1003
- River Plate: 1202
- Colon: 1017
- Newells: 753
- Belgrano: 1410
- Boca Juniors: 1755

Media de los km recorridos:
1214.25

Varianza de los km recorridos por todos los equipos:


872.78